In [4]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [46]:
# Simulation parameters 
N = 1024
D = 5e-3
dx = D / N
wl = 810e-9
sig = 50e-6
L = 50e-3
focal_length = 20e-3  # TODO: a bit unrealistic 

# X, Y range centered around 0
X = (np.arange(1, N + 1) - (N / 2 + 0.5)) * dx
XX, YY = np.meshgrid(X, X)

# k-space grid
df = 1 / D
freqs = df * np.arange(-N // 2, N // 2)
freq_XXs, freq_YYs = np.meshgrid(freqs, freqs)
light_k = 2 * np.pi / wl
k_xx = freq_XXs * 2 * np.pi
k_yy = freq_YYs * 2 * np.pi
X_crystal_plane = wl * focal_length * freqs
XX_crystal, YY_crystal = np.meshgrid(X_crystal_plane, X_crystal_plane)
dx_crystal = X_crystal_plane[1] - X_crystal_plane[0]

k_z_sqr = light_k ** 2 - (k_xx ** 2 + k_yy ** 2)
# Remove all the negative component, as they represent evanescent waves, see Fourier Optics page 58
np.maximum(k_z_sqr, 0, out=k_z_sqr)
k_z = np.sqrt(k_z_sqr)


In [47]:
from colorsys import hls_to_rgb

def colorize(z, theme='dark', saturation=1., beta=1.4, transparent=False, alpha=1., max_threshold=1.):
    r = np.abs(z)
    r /= max_threshold * np.max(np.abs(r))
    arg = np.angle(z)

    h = (arg + np.pi) / (2 * np.pi) + 0.5
    l = 1. / (1. + r ** beta) if theme == 'white' else 1. - 1. / (1. + r ** beta)
    s = saturation

    c = np.vectorize(hls_to_rgb)(h, l, s)  # --> tuple
    c = np.array(c)  # -->  array of (3,n,m) shape, but need (n,m,3)
    c = c.swapaxes(0, 2)
    if transparent:
        a = 1. - np.sum(c ** 2, axis=-1) / 3
        alpha_channel = a[..., None] ** alpha
        return np.concatenate([c, alpha_channel], axis=-1)
    else:
        return c

def show(E, ax, title, use_colorize=True):
    if use_colorize:
        ax.imshow(colorize(E))
    else:
        I = np.abs(E)**2
        imm = ax.imshow(I)
        ax.figure.colorbar(imm, ax=ax)
    ax.set_title(title)
    
def ft2(E):
    return np.fft.fftshift(np.fft.fft2(np.fft.fftshift(E))) * D

def get_diffuser(M_macro_pixels=128):
    phi_small = np.random.uniform(0, 2*np.pi, size=(M_macro_pixels, M_macro_pixels))
    block_size = N // M_macro_pixels
    diffuser_phi = np.kron(phi_small, np.ones((block_size, block_size)))
    return diffuser_phi

In [48]:
def propagate(diffuser_phi, incidence_angle, pump_waist=20000e-6, debug_mode=False):
    # Assume I began somewhere at the first detector, and reached the diffuser with this size 
    sig = 0.35e-3
    E_at_diffuser = np.exp(-(XX**2 + YY**2)/sig**2)
    # Angle theta so e^(i*k_x*x), with k_x=2pi/wl * sin(theta) 
    angle_factor = (2 * np.pi / wl) * np.sin(incidence_angle)

    E_at_diffuser_angled = E_at_diffuser * np.exp(1j*XX*angle_factor)
    
    E_after_diffuser = E_at_diffuser_angled * np.exp(1j*diffuser_phi)
    
    # Crystal at farfield, so the filter incurred by the pump will not be diagonal in the diffuser plane, 
    # and will result with a finite memory 
    E_at_crystal = ft2(E_after_diffuser)
    pump = np.exp(-(XX_crystal**2 + YY_crystal**2)/pump_waist**2)
    E_crystal_filtered = E_at_crystal * pump
    
    E_before_diffuser_again = ft2(E_crystal_filtered)
    E_after_diffuser_again = E_before_diffuser_again * np.exp(1j*diffuser_phi)
    
    # Not allowed to use the reangle trick, since if pump is very tight - we will completely forget the original angle!  
    # E_reangled = E_after_diffuser_again * np.exp(1j*XX*angle_factor)
    
    E_end = ft2(E_after_diffuser_again)
    
    if debug_mode:
        fig, axes = plt.subplots(2, 4)
        show(E_at_diffuser_angled, axes[0, 0], 'E at diffuser angled')
        show(E_after_diffuser, axes[0, 1], 'E after diffuser')
        show(E_at_crystal, axes[0, 2], 'E at crystal')
        show(pump, axes[0, 3], 'pump')
        show(E_crystal_filtered, axes[1, 0], 'E crystal filtered')
        show(E_before_diffuser_again, axes[1, 1], 'E before diffuser again')
        show(E_after_diffuser_again, axes[1, 2], 'E after diffuser again')
        show(E_end, axes[1, 3], 'End')
        fig.show()
    
    return E_end 
        


In [103]:
from scipy.ndimage import shift as imshift
from skimage.registration import phase_cross_correlation


def visualize_angles(diffuser_phi, angles, pump_waist, zoom_in=450):
    E_ends = [] 
    for incidence_angle in angles:
        E_end = propagate(diffuser_phi, incidence_angle, pump_waist)
        E_ends.append(E_end)
    N_angles = len(angles)
    fig, ax = plt.subplots(1, N_angles, figsize=(16, 3.4))
    fig.suptitle(f'pump_waist={pump_waist*1e6:.1f} um')
    for i in range(N_angles):
        show(E_ends[i], ax[i], f'E end angle={angles[i]:.2f}', use_colorize=False)
        ax[i].set_xlim(zoom_in, N-zoom_in)
        ax[i].set_ylim(zoom_in, N-zoom_in)
    fig.show()    
    return E_ends


def get_PCCs(diffuser_phi, angles, pump_waist, D_area, debug_mode=False):
    E_end0 = propagate(diffuser_phi, 0, pump_waist)
    I_end0 = np.abs(E_end0)**2
    
    hD = D_area // 2
    mask = np.index_exp[N//2 - hD: N//2 + hD, 
                        N//2 - hD: N//2 + hD]
        
    PCCs_at0, PCCs_shifted_phys, PCCs_max = [], [], []
    PCCs_actual_shift_x = [] 
    for incidence_angle in angles:
        E_end = propagate(diffuser_phi, incidence_angle, pump_waist)
        I_end = np.abs(E_end)**2
        
        # 1) PCC with no shift - for single speckle grain size pump 
        pcc0 = np.corrcoef(I_end0[mask].ravel(), I_end[mask].ravel())[0, 1]
        
        
        # 2) Offset due to angle - same speckles but translated 
        fx = np.sin(incidence_angle) / wl   # k_x = k*sin(theta)
        shift_px = fx / df  # from frequency units to pixel units
        shifted_I_end = imshift(I_end, shift=(0, shift_px), order=1, mode='constant', prefilter=False)
        pcc_phys = np.corrcoef(I_end0[mask].ravel(), shifted_I_end[mask].ravel())[0, 1]
        
        # 3) Max PCC via phase correlation (subpixel)
        (row_s, col_s), _, _ = phase_cross_correlation(I_end0, I_end, upsample_factor=16)
        I_max = imshift(I_end, shift=(row_s, col_s), order=1, mode='constant', prefilter=False)
        pcc_max = np.corrcoef(I_end0[mask].ravel(), I_max[mask].ravel())[0, 1]
                
        
        PCCs_at0.append(pcc0)
        PCCs_shifted_phys.append(pcc_phys)
        PCCs_max.append(pcc_max)
        PCCs_actual_shift_x.append(col_s)
        
    if debug_mode:
        # I might have a bug here with where I add the patch 
        fig, ax = plt.subplots()
        show(I_end, ax, f'I_end0', use_colorize=False)
        rect = patches.Rectangle(
                (N//2 - D_area//2 - round(shift_px), N//2 - D_area//2),  # Bottom-left corner
                D_area, D_area,  # Width, Height
                linewidth=0.5,  # thin
                edgecolor='white',
                facecolor='none',
                linestyle='dashed'
            )
        ax.add_patch(rect)
        fig.show()
        
    return PCCs_shifted_phys, PCCs_at0, PCCs_max, PCCs_actual_shift_x

In [111]:
# Debug propagation
diffuser_phi = get_diffuser(M_macro_pixels=128)
_ = propagate(diffuser_phi, incidence_angle=0.01, pump_waist=100e-6, debug_mode=True)

In [115]:
# Visualize experiment 
diffuser_phi = get_diffuser(M_macro_pixels=128)
incidence_angles = np.linspace(0.0, 0.03, 5)
# E_ends = visualize_angles(diffuser_phi, incidence_angles, pump_waist=7e-6, zoom_in=470)    # ~single mode filter 
# E_ends = visualize_angles(diffuser_phi, incidence_angles, pump_waist=100e-6, zoom_in=350)  # stays in place with different speckle 
# E_ends = visualize_angles(diffuser_phi, incidence_angles, pump_waist=300e-6, zoom_in=300)  # some stays and some moves 
# E_ends = visualize_angles(diffuser_phi, incidence_angles, pump_waist=500e-6, zoom_in=300)    # mostly moves, some DC stays 
# E_ends = visualize_angles(diffuser_phi, incidence_angles, pump_waist=800e-6, zoom_in=300)    # moves ~classical. slight DC component 
E_ends = visualize_angles(diffuser_phi, incidence_angles, pump_waist=4500e-6, zoom_in=200)    # ~no DC component    

In [106]:
# Memory for different pump waists 
diffuser_phi = get_diffuser(M_macro_pixels=128)
incidence_angles = np.linspace(0.0, 0.03, 20)  # 0.03 is a good maximal angle, keeping reasonable resolution for linear phase, and 
                                               # and there is some light close to the optical axis for diffuser with M=128 
pump_waists = np.array([7, 100, 200, 300, 400, 600, 800, 1200])*1e-6
# pump_waists = np.array([7, 300, 1500])*1e-6
all_PCCs0 = []
all_PCCs_shifted = []
all_PCCs_max = []
shifts_max_PCC = []
for pump_waist in pump_waists:
    PCCs_shifted_phys, PCCs_at0, PCCs_max, PCCs_actual_shift_x = get_PCCs(diffuser_phi, incidence_angles, pump_waist, D_area=50, debug_mode=False)
    all_PCCs_shifted.append(PCCs_shifted_phys)
    all_PCCs0.append(PCCs_at0)
    all_PCCs_max.append(PCCs_max)
    shifts_max_PCC.append(PCCs_actual_shift_x)

In [112]:
def plot_pccs(angles, pccs, title='PCCs'):
    fig, ax = plt.subplots()
    for i, pump_waist in enumerate(pump_waists):
        angles_deg = (angles / (2*np.pi)) * 360 
        ax.plot(angles_deg, pccs[i], '.-', label=f'$\sigma_{{p}}$={pump_waist*1e6:.0f}$\mu$m')
        ax.set_ylabel('PCC')
        ax.set_xlabel('angle (deg)')
        ax.legend()
        ax.set_title(title)
    fig.show()


def plot_best_shifts(angles, shifts, title=''):
    fig, ax = plt.subplots()
    for i, pump_waist in enumerate(pump_waists):
        angles_deg = (angles / (2*np.pi)) * 360 
        ax.plot(angles_deg, shifts[i], '.-', label=f'$\sigma_{{p}}$={pump_waist*1e6:.0f}$\mu$m')
        ax.set_ylabel('best x shift')
        ax.set_xlabel('angle (deg)')
        ax.legend()
        ax.set_title(title)
    fig.show()


plot_pccs(incidence_angles, all_PCCs0, title=f'PCCs on axis')
plot_pccs(incidence_angles, all_PCCs_shifted, title=f'PCCs shifted')
plot_pccs(incidence_angles, all_PCCs_max, title=f'PCCs max')

plot_best_shifts(incidence_angles, shifts_max_PCC)

In [101]:
shifts_max_PCC

[[0.0, -0.375, 0.1875, 0.1875, 0.125, 0.3125, 0.25, 0.5, 0.5, -0.3125],
 [0.0,
  20.5,
  41.0625,
  61.5625,
  82.0625,
  102.625,
  123.125,
  143.6875,
  164.25,
  57.0],
 [0.0,
  20.5625,
  41.125,
  61.75,
  82.3125,
  102.875,
  123.4375,
  144.0,
  164.5625,
  185.125]]